In [35]:
# session.write_pandas(df,table_name="HR_CHURN",auto_create_table=True).show()
# !pip install fosforml

In [3]:
import pandas as pd
df = pd.read_csv("HR_CHURN.csv",index_col='Unnamed: 0')

from snowflake.snowpark import Session
		
CONNECTION_PARAMETERS = {
"account": "RWB76633",
"user":"MAHESHG",
"password": "Jamesbond@123",
"role": "MAHESHG",
"database": "NVIDIA_DB",
"warehouse": "FOSFOR_WAREHOUSE",
"schema": "NVIDIA_SCHEMA", }
    
session = Session.builder.configs(CONNECTION_PARAMETERS).create()

In [10]:
def is_table_exists(session,MODEL_MONITOR):
    query = f"""
    SHOW TABLES LIKE '{MODEL_MONITOR}';
    """
    result = session.sql(query).collect()
    return len(result) > 0

In [12]:
is_table_exists(session,'MODEL_MONITORING')

True

In [4]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier,RandomForestClassifier,RandomForestRegressor
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [5]:
snow_df = session.create_dataframe(df)
snow_df = snow_df.drop(['Unnamed: 0'])
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
test_df = apply_label_encoding(test_df)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']

In [6]:
pipeline = GradientBoostingClassifier(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)
pred_snow_df = pipeline.predict(test_df)

Got error object of type 'NoneType' has no len() when trying to read default values from function: <function SnowparkModelTrainer._build_fit_wrapper_sproc.<locals>.fit_wrapper_function at 0x7f456e607af0>. Proceeding without creating optional arguments
The version of package 'snowflake-snowpark-python' in the local environment is 1.21.1, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different betw

In [9]:
from fosforml import register_model

register_model(
  model_obj=pipeline,
  session=session,
  name="TestModelForDrifts",
  snowflake_df=pred_snow_df,
  dataset_name="HR_CHURN",
  dataset_source="Dataset",
  source="NOTEBOOK",
  description="This is a Snowflake model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

SnowparkSQLException: (1300) (1304): 01b6e158-0002-e346-0033-ac8706f1234e: 002043 (02000): SQL compilation error:
Object does not exist, or operation cannot be performed.